## Para iniciar as análises e preparação para a modelagem, vamos entender e preparar os datasets para criar uma base de treino e teste para os modelos preditivos.

### Começamos improtando todas as bibliotecas necessárias para esse trabalho.

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Vamos retirar a restrição de limite de colunas visíveis para ficar mais fácil visualizar as features.
pd.set_option('display.max_columns', None)

In [2]:
# Vamos abrir as tabelas .csv extraídas da base de dados do app como dataframes de trabalho.
users = pd.read_csv('../CSVs/tbl_users.csv', sep=';', low_memory=False)
events = pd.read_csv('../CSVs/tbl_events.csv', low_memory=False)
users_events = pd.read_csv('../CSVs/tbl_users_events.csv', low_memory=False)
analise = pd.read_csv('../CSVs/gg_analise_db1.csv', sep=';', low_memory=False)
users_games = pd.read_csv('../CSVs/tbl_users_games.csv', low_memory=False)
games = pd.read_json('../CSVs/tbl_games.json')
friends = pd.read_csv('../CSVs/tbl_friends.csv', low_memory=False)
chats = pd.read_csv('../CSVs/tbl_chat_members.csv', low_memory=False)
plataformas = pd.read_csv('../CSVs/tbl_platforms.csv', low_memory=False)
platforms = pd.read_csv('../CSVs/tbl_users_platforms.csv', low_memory=False)
logs = pd.read_csv('../CSVs/tbl_users_login_log.csv', low_memory=False)

### Vamos começar o trabalho de organização, limpeza e engenharia de features.

In [3]:
# Ao tentar fazer a mudança para INT, apareceram erros devido a alguns dados da coluna não serem numéricos.
# Aí eu busquei esses registros, que eram poucos, para eliminar as linhas.
users.loc[users['id']=='   "']
users.loc[users['id']=='?>"']

# Eliminando as linhas problemáticas.
users.drop(index=16790, inplace=True)
users.drop(index=22557, inplace=True)

# Aí, sim, alterando a coluna id para INT.
users.id = users.id.astype(int)

In [4]:
# Definindo o ID como index e renomeando para 'user_id'.
gg_users = users.set_index('id', drop=True)
gg_users.rename_axis('user_id', inplace=True)

### Agora, vamos trabalhar algumas features em diferentes tabelas para criar as métricas que vamos utilizar para tentar identificar comportamentos dos usuários no app.

In [5]:
# Vamos agrupar os jogos por user_id para calcular a quantidade de jogos que
# cada usuário tem cadastrado no app e preparar para concatenar ao dataset principal.
grouped_games = users_games.groupby(by='user_id').count()
grouped_games.rename(columns={'id':'qtde_games'}, inplace=True)
grouped_games.drop(columns=['game_id', 
                            'platform_id', 
                            'network_id', 
                            'is_favorite', 
                            'status_id'], 
                   inplace=True
                  )

In [6]:
# Vamos fazer o mesmo para a quantidade de Grupos criados ('events').
events.rename(columns={'created_by':'user_id'}, inplace=True)
grouped_events = events.groupby(by='user_id').count()
grouped_events.rename(columns={'id':'qtde_grupos'}, inplace=True)
grouped_events.drop(grouped_events.iloc[:, 1:], inplace = True, axis = 1) 

In [7]:
# E o mesmo para a quantidade de conexões ('friends').
grouped_friends = friends.groupby(by='user_id').count()
grouped_friends.drop(columns=['id'], inplace=True)
grouped_friends.rename(columns={'friend_id':'qtde_contatos'}, inplace=True)

In [8]:
# Também para as ações realizadas pelos usuários no app ('users_events'). 
grouped_user_events = users_events.groupby(by='user_id').count()
grouped_user_events.rename(columns={'id':'qtde_eventos'}, inplace=True)
grouped_user_events.drop(grouped_user_events.iloc[:, 1:], inplace = True, axis = 1) 

In [9]:
# E também para as mensagens de chat trocadas por cada usuário ('chats').
chats_grouped = chats.groupby(by='user_id').count()
chats_grouped.rename(columns={'chat_id':'qtde_mensagens'}, inplace=True)
chats_grouped.drop(chats_grouped.iloc[:, 1:], inplace = True, axis = 1) 

In [10]:
# Tambem para a quantidade de Plataformas cadastradas ('platforms').
platforms_grouped = platforms.groupby(by='user_id').count()
platforms_grouped.rename(columns={'id':'qtde_plataformas'}, inplace=True)
platforms_grouped.drop(platforms_grouped.iloc[:, 1:], inplace = True, axis = 1) 

In [11]:
# E, por fim, para a quantidade de logins realizados pelo usuários, o que mostra a frequência de acessos ('logs').
grouped_logs = logs.groupby(by='user_id').count()
grouped_logs.rename(columns={'id':'qtde_acessos'}, inplace=True)
grouped_logs.drop(columns=['login_datetime'], inplace=True)

In [12]:
# Agora concatenamos todas as features junto com o dataset principal.
users_gg = pd.concat([gg_users, 
                      grouped_games, 
                      grouped_events, 
                      grouped_friends, 
                      grouped_user_events, 
                      chats_grouped, 
                      platforms_grouped, 
                      grouped_logs], 
                      axis = 1
                    )

In [13]:
# Os dados nulos precisam ser entendidos pelo modelo, então vamos tratá-los como 0, 
# pois significa que o usuário não tem nenhuma daquelas features cadastradas.
users_gg["qtde_games"].fillna(0, inplace=True)
users_gg["qtde_grupos"].fillna(0, inplace=True)
users_gg["qtde_contatos"].fillna(0, inplace=True)
users_gg["qtde_eventos"].fillna(0, inplace=True)
users_gg["qtde_mensagens"].fillna(0, inplace=True)
users_gg["qtde_plataformas"].fillna(0, inplace=True)
users_gg["qtde_acessos"].fillna(0, inplace=True)

In [14]:
# Eliminando as linhas que não têm informação de last_login. Não serão úteis.
users_gg.dropna(how='any', subset=['last_login'], inplace=True)

In [15]:
# Ao tentar converter creation_datetime, que é string, para o formato datetime, 
# deu erro porque alguma linha tinha valor '0000-00-00 00:00:00'.
# Precisamos encontrar essa(s) linha(s) e eliminá-la(s).
users_gg.loc[users_gg['creation_datetime']=='0000-00-00 00:00:00']

,email,pass_hash,name,nickname,avatar,profile_pic_id,uploaded_photo_timestamp,top_image,profile_bg_id,birthday,gender,city,state,country,ip_country_code,lat,lng,fbid,steam_uid,steam_pic_url,quickblox_id,steam_id,psn_id,xboxlive_id,nintendo_id,gamecenter_id,googleplay_id,battlenet_id,uplay_id,origin_id,youtube_id,twitch_id,phone_country_prefix,phone_country,phone_area,phone_number,creation_datetime,ak_id,ak_number,update_datetime,remember_token,access_token,firebase_id,special_mark,last_login,last_interaction,qtde_games,qtde_grupos,qtde_contatos,qtde_eventos,qtde_mensagens,qtde_plataformas,qtde_acessos
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
49211,NaN,NaN,NaN,deletethis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0000-00-00 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-09-03 20:17:30,2020-09-03 20:17:30,0.0,0.0,0.0,0.0,0.0,1.0,4.0


In [16]:
users_gg.drop(labels=49211, inplace=True)

In [17]:
# Agora vamos converter todos os timestamps para datetime.
users_gg['creation_datetime'] = pd.to_datetime(users_gg['creation_datetime'])
users_gg['last_login'] = pd.to_datetime(users_gg['last_login'])
users_gg['last_interaction'] = pd.to_datetime(users_gg['last_interaction'])

# Depois vamos criar uma coluna de churn, com base no número de dias de inatividade 
# entre hoje e a data de último login. Se for maior que 30 dias, consideramos churn.
# Essa coluna será o nosso targets para os modelos.
# Definimos 30 dias de acordo com o que a área de negócios considera um período 
# indicativo de parada de uso do app.
curr_time = pd.to_datetime("now")
users_gg['inatividade'] = curr_time - users_gg['last_interaction']
users_gg['inatividade_days'] = users_gg['inatividade'].dt.days
users_gg['churn'] = users_gg['inatividade_days']>30

In [18]:
# Agora vamos criar uma coluna que nos diga quanto tempo o usuário passou dentro do app 
# entre o primeiro login 'creation_datetime' e a última interação 'last_interaction'.
# E vamos criar outra coluna identificando aqueles que ficaram menos de 30 dias no app.
users_gg['tempo_uso'] = users_gg['last_interaction'] - users_gg['creation_datetime']
users_gg['tempo_uso_days'] = users_gg['tempo_uso'].dt.days
users_gg['pouco_uso'] = users_gg['tempo_uso_days']<30

In [19]:
users_gg['pouco_uso'].value_counts()

False    55840
True     15348
Name: pouco_uso, dtype: int64

### Vemos que cerca de 20% dos usuários tiveram menos de 30 dias de uso do app.
### 

### Agora, como a empresa começou a fazer modificações no app e iniciou ações de marketing para atração de novos usuários a partir de Julho de 2020, definimos com a área de negócios que vamos trabalhar com os usuários que entraram a partir dessa data apenas, desconsiderando os usuários antigos.

In [20]:
gg_dataset = users_gg.loc[users_gg['creation_datetime']>'2020-07-01']

### Analisando as features nesse período, vemos que há muitas sem qualquer valor e uma grande quantidade delas são strings, que precisarão ser convertidas em números para a análise dos modelos.
### Vamos eliminar as nulas e ver como tratar as incompletas.

In [21]:
gg_dataset.dropna(how='all', axis=1, inplace=True)

<ipython-input-21-d198368bc5f5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gg_dataset.dropna(how='all', axis=1, inplace=True)


In [22]:
gg_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20120 entries, 51076 to 71195
Data columns (total 49 columns):
 #   Column                    Non-Null Count  Dtype          
---  ------                    --------------  -----          
 0   email                     18882 non-null  object         
 1   pass_hash                 6800 non-null   object         
 2   name                      10906 non-null  object         
 3   nickname                  17748 non-null  object         
 4   avatar                    11229 non-null  object         
 5   profile_pic_id            11152 non-null  float64        
 6   uploaded_photo_timestamp  740 non-null    object         
 7   top_image                 1074 non-null   object         
 8   profile_bg_id             900 non-null    object         
 9   city                      590 non-null    object         
 10  ip_country_code           19933 non-null  object         
 11  lat                       14202 non-null  float64        
 12  

In [23]:
# Algumas features não nos serão úteis pois contém informações irrelevantes para as análises.
# Vamos eliminá-las.
gg_dataset_work = gg_dataset.drop(columns=['pass_hash', 
                                           'uploaded_photo_timestamp', 
                                           'top_image', 
                                           'profile_bg_id', 
                                           'phone_country', 
                                           'update_datetime', 
                                           'remember_token', 
                                           'access_token', 
                                           'special_mark', 
                                           'firebase_id']
                                 )

In [24]:
# agora, pra facilitar as análises numéricas, vamos transformas algumas colunas em categóricas.
# Por exemplo, vamos considerar apenas a presença ou não de avatar, sem especificar qual.
# Da mesma forma, a presença ou não de ID para as plataformas, sem considerar o ID específico.
gg_dataset_work['email'] = np.where(gg_dataset_work['email'].isnull(), 0, 1)
gg_dataset_work['name'] = np.where(gg_dataset_work['name'].isnull(), 0, 1)
gg_dataset_work['nickname'] = np.where(gg_dataset_work['nickname'].isnull(), 0, 1)
gg_dataset_work['avatar'] = np.where(gg_dataset_work['avatar'].isnull(), 0, 1)
gg_dataset_work['profile_pic_id'] = np.where(gg_dataset_work['profile_pic_id'].isnull(), 0, 1)
gg_dataset_work['city'] = np.where(gg_dataset_work['city'].isnull(), 0, 1)
gg_dataset_work['ip_country_code'] = np.where(gg_dataset_work['ip_country_code']=='BR', 1, 0)
gg_dataset_work['fbid'] = np.where(gg_dataset_work['fbid'].isnull(), 0, 1)
gg_dataset_work['quickblox_id'] = np.where(gg_dataset_work['quickblox_id'].isnull(), 0, 1)
gg_dataset_work['steam_id'] = np.where(gg_dataset_work['steam_id'].isnull(), 0, 1)
gg_dataset_work['psn_id'] = np.where(gg_dataset_work['psn_id'].isnull(), 0, 1)
gg_dataset_work['xboxlive_id'] = np.where(gg_dataset_work['xboxlive_id'].isnull(), 0, 1)
gg_dataset_work['nintendo_id'] = np.where(gg_dataset_work['nintendo_id'].isnull(), 0, 1)
gg_dataset_work['gamecenter_id'] = np.where(gg_dataset_work['gamecenter_id'].isnull(), 0, 1)
gg_dataset_work['googleplay_id'] = np.where(gg_dataset_work['googleplay_id'].isnull(), 0, 1)
gg_dataset_work['battlenet_id'] = np.where(gg_dataset_work['battlenet_id'].isnull(), 0, 1)
gg_dataset_work['uplay_id'] = np.where(gg_dataset_work['uplay_id'].isnull(), 0, 1)
gg_dataset_work['origin_id'] = np.where(gg_dataset_work['origin_id'].isnull(), 0, 1)
gg_dataset_work['youtube_id'] = np.where(gg_dataset_work['youtube_id'].isnull(), 0, 1)
gg_dataset_work['twitch_id'] = np.where(gg_dataset_work['twitch_id'].isnull(), 0, 1)
gg_dataset_work['phone_number'] = np.where(gg_dataset_work['phone_number'].isnull(), 0, 1)

In [25]:
gg_dataset_work['lng'] = gg_dataset_work['lng'].astype('float64')

In [26]:
gg_dataset_work.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20120 entries, 51076 to 71195
Data columns (total 39 columns):
 #   Column             Non-Null Count  Dtype          
---  ------             --------------  -----          
 0   email              20120 non-null  int64          
 1   name               20120 non-null  int64          
 2   nickname           20120 non-null  int64          
 3   avatar             20120 non-null  int64          
 4   profile_pic_id     20120 non-null  int64          
 5   city               20120 non-null  int64          
 6   ip_country_code    20120 non-null  int64          
 7   lat                14202 non-null  float64        
 8   lng                14202 non-null  float64        
 9   fbid               20120 non-null  int64          
 10  quickblox_id       20120 non-null  int64          
 11  steam_id           20120 non-null  int64          
 12  psn_id             20120 non-null  int64          
 13  xboxlive_id        20120 non-null  int64  

In [27]:
# Creating a dataframe with 30% registers of original dataframe 
gg_30 = gg_dataset_work.sample(frac = 0.3) 
  
# Creating dataframe with the rest of the 70% registers 
gg_70 = gg_dataset_work.drop(gg_30.index) 
  
print("\n30% of the givem DataFrame:") 
print(gg_30.info()) 
  
print("\nrest 70% of the given DataFrame:") 
print(gg_70.info())


30% of the givem DataFrame:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6036 entries, 68989 to 69503
Data columns (total 39 columns):
 #   Column             Non-Null Count  Dtype          
---  ------             --------------  -----          
 0   email              6036 non-null   int64          
 1   name               6036 non-null   int64          
 2   nickname           6036 non-null   int64          
 3   avatar             6036 non-null   int64          
 4   profile_pic_id     6036 non-null   int64          
 5   city               6036 non-null   int64          
 6   ip_country_code    6036 non-null   int64          
 7   lat                4222 non-null   float64        
 8   lng                4222 non-null   float64        
 9   fbid               6036 non-null   int64          
 10  quickblox_id       6036 non-null   int64          
 11  steam_id           6036 non-null   int64          
 12  psn_id             6036 non-null   int64          
 13  xboxlive_id   

In [28]:
gg_train = gg_70.copy()

In [29]:
gg_test = gg_30.copy()
gg_test.drop(columns=['churn'], inplace=True)

In [30]:
gg_train.to_csv('../CSVs/games_train.csv')

In [31]:
gg_test.to_csv('../CSVs/games_test.csv')